In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Call data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 999

In [2]:
df = pd.read_csv('/Users/allen/Downloads/call_data.csv')

# column Arrived Time ends with :000 which needs to be removed
df['Arrived Time'] = df['Arrived Time'].str.replace(':000', '')

In [4]:
%%time
# change datatype to datetime
df['Arrived Time'] = pd.to_datetime(df['Arrived Time'])

CPU times: user 7min 20s, sys: 1.83 s, total: 7min 22s
Wall time: 7min 25s


In [5]:
%%time
# change datatype to datetime
df['Original Time Queued'] = pd.to_datetime(df['Original Time Queued'])

CPU times: user 6min 53s, sys: 2.7 s, total: 6min 55s
Wall time: 7min 1s


In [8]:
# Calculate respnse time (minute)
df['response_time'] = (df['Arrived Time'] - df['Original Time Queued']) / np.timedelta64(1,'m')
df['response_time'] = df['response_time'].apply(lambda x: round(x, ndigits=2))

# filter unuseful response time
df = df[(df['response_time'] > 1) & (df['response_time'] < 60)]

# create year month to aggregate
df['YearMonth'] = pd.to_datetime(df['Original Time Queued'].map(lambda x: str(x.year) + '-' + str(x.month)))

# output filtered data
df.to_csv('/Users/allen/Documents/Data_512/Data/call_data_filtered.csv', index=False)

df['Year'] = df['YearMonth'].dt.year

beat_agg = df.groupby(['Year','Beat']).agg(
    crime_counts = ('response_time', 'count'),
    avg_response_time = ('response_time', 'mean')
).sort_values('avg_response_time').reset_index()

In [41]:
beat_agg.head()

,Year,Beat,crime_counts,avg_response_time
0,2010,CC,1,1.07
1,2014,SD,1,1.12
2,2018,DSPVD,1,1.25
3,2014,HBR,1,1.32
4,2012,CMD4,1,1.48


In [42]:
beat_agg.to_csv('/Users/allen/Documents/Data_512/Data/beat_cc_resp_time.csv', index=False)

# Crime data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 999

In [43]:
df = pd.read_csv('/Users/allen/Downloads/crime_data.csv')

df.dropna(inplace=True)

# need datetime to extract year info
df['Year'] = pd.to_datetime(df['Occ Date']).dt.year

# calculate crime aggregates
crime_agg = df.groupby(['Year','BEAT','Crime Group','Crime Category']).agg(
    crime_counts = ('Definition', 'count')
).reset_index()

In [45]:
crime_agg.head()

,Year,BEAT,Crime Group,Crime Category,crime_counts
0,2008,B1,Person Crime,Aggravated Assault,23
1,2008,B1,Person Crime,Homicide,1
2,2008,B1,Person Crime,Rape,2
3,2008,B1,Person Crime,Robbery,20
4,2008,B1,Property Crime,Burglary,127


In [46]:
crime_agg.to_csv('/Users/allen/Documents/Data_512/Data/crime_data_agg.csv', index=False)